# 回測

## 夏普率及最大虧損

### 夏普率

夏普率的定義：

$$訊息比率=\frac{超額報酬平均值}{超額報酬標準差}$$

其中，

$$超額報酬 = 投資組合報酬 - 比較基準報酬$$

In [ ]:
%matplotlib inline

# Pandas v0.16 之前，使用下面這個
#import pandas.io.data as web

# Pandas v0.16 之後，使用下面這個
import pandas_datareader.data as web

from pandas import Series, DataFrame
import numpy as np

from datetime import datetime

In [ ]:
# 讀取從指定日期之後的股價資訊
df=web.DataReader("TSLA", 'yahoo', datetime(2016,1,1))

In [ ]:
# 手動設定進出場日期
# 進出場日期之後會由策略自動產生

#進場日期
inDate = datetime(2016,2,10)
#出場日期
outDate = datetime(2016,4,6)

In [ ]:
dailyRet = df['Close'][inDate : outDate].pct_change()

In [ ]:
dailyRet.head()

In [ ]:
dailyRet.tail()

In [ ]:
#假設無風險利率為 4%
#假設一年有252個交易日
excessRet = dailyRet - 0.04/252

sharpeRatio = np.sqrt(252.0)*np.mean(excessRet)/np.std(excessRet)

In [ ]:
sharpeRatio

### 最大虧損額及最大虧損持續時間

### 下圖取自 Qunatitative Trading by Ernest Chan 
<img src="images/maxdd.png" width="800" />

In [ ]:
cumRet = np.cumprod(1 + excessRet) - 1

In [ ]:
type(cumRet)

In [ ]:
# maxDD: maximum drawdown
# maxDDD: maximum drawdown duration

def calculateMaxDD(cumRet):
    highwatermark = np.zeros(np.size(cumRet))
    drawdownduration = np.zeros(np.size(cumRet))
    drawdown = np.zeros(np.size(cumRet))
    for t in range(2, cumRet.size):
        highwatermark[t] = max(highwatermark[t-1], cumRet[t])
        drawdown[t] = (1 + highwatermark[t]) / (1 + cumRet[t]) - 1
        if (drawdown[t] == 0):
            drawdownduration[t] = 0
        else:
            #drawdownduration[t] = drawdownduration[t-1] + 1
            # 底下這一行把沒有交易的天數也都算進來，這樣感覺比較實際
            drawdownduration[t] = drawdownduration[t-1] + (cumRet.index[t]-cumRet.index[t-1]).days
    maxDD = max(drawdown)
    maxDDD = max(drawdownduration)
    Series(drawdownduration, index=cumRet.index).plot()
    return maxDD, maxDDD

In [ ]:
calculateMaxDD(cumRet)

In [ ]:
cumRet.plot(style='ro-')